# Chat Models

## 1_chat_model_basic

In [1]:
%pip install -q python-dotenv langchain langchain-community langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

In [3]:
# Load environment variables from .env
load_dotenv()

True

In [4]:
from langchain_ollama.llms import OllamaLLM

In [5]:
# Create a OllamaLLM model
model = OllamaLLM(base_url="http://localhost:11434", model="llama3.2", temperature=0)

In [6]:
# Invoke the model with a message
result = model.invoke("What is 81 divided by 9?")

In [7]:
print("Full result:")
print(result)

Full result:
81 ÷ 9 = 9


## 2_chat_model_basic_conversation

In [7]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [8]:
from langchain_ollama import ChatOllama

In [9]:
model = ChatOllama(
        base_url="http://localhost:11434",
        model="llama3.2",
        temperature=0)

In [10]:
# SystemMessage:
#   Message for priming AI behavior, usually passed in as the first of a sequenc of input messages.
# HumanMessagse:
#   Message from a human to the AI model.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

In [11]:
# Invoke the model with messages
result = model.invoke(messages)

In [12]:
print(f"Answer from AI: {result}")

Answer from AI: content='81 ÷ 9 = 9' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-03T20:23:33.185247555Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1541235433, 'load_duration': 28559737, 'prompt_eval_count': 40, 'prompt_eval_duration': 760769000, 'eval_count': 9, 'eval_duration': 628613000} id='run-3ef2ca98-3946-4a16-87f5-d6d23f296dd9-0' usage_metadata={'input_tokens': 40, 'output_tokens': 9, 'total_tokens': 49}


In [13]:
print(f"Answer from AI: {result.content}")

Answer from AI: 81 ÷ 9 = 9


In [33]:
# AIMessage:
#   Message from an AI.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
]

In [34]:
# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

Answer from AI: 10 × 5 = 50


In [37]:
# Invoke the model with messages
result = model.invoke("Summarize our conversation so far")
print(f"Answer from AI: {result.content}")

Answer from AI: We haven't had a conversation yet. This is the first message in our conversation. What would you like to talk about? I can summarize our conversation at the end if you'd like.


In [38]:
# AIMessage:
#   Message from an Human.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
    AIMessage(content="10 times 5 is 50."),
    HumanMessage(content="Summarize our conversation so far")
]

In [39]:
# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

Answer from AI: We've had a simple math conversation so far. You asked me two questions:

1. To calculate the result of dividing 81 by 9, which I answered as 9.
2. To calculate the result of multiplying 10 by 5, which I answered as 50.

That's the summary of our conversation so far!


### 3_chat_model_alternatives

In [21]:
from langchain_openai import ChatOpenAI

In [28]:
openai_model = ChatOpenAI(model="gpt-4o-mini")

In [29]:
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

In [30]:
# Invoke the model with messages
result = openai_model.invoke(messages)
print(f"Answer from OpenAI: {result.content}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## 4_chat_model_conversation_with_user

In [14]:
# Use a list to store messages
chat_history = []

In [15]:
# Set an initial system message (optional)
system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)  # Add system message to chat history

In [16]:
# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))  # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))  # Add AI message

    print(f"AI: {response}")


print("---- Message History ----")
print(chat_history)

You:  exit


---- Message History ----
[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}, response_metadata={})]


## 5_chat_model_conversation_memory

In [34]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [35]:
# store is a dictionary that maps session IDs to their corresponding chat histories.
store = {}  # memory is maintained outside the chain

In [36]:
# A function that returns the chat history for a given session ID.
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [37]:
#  Define a RunnableConfig object, with a `configurable` key. session_id determines thread
config = {"configurable": {"session_id": "1"}}

In [39]:
conversation = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [40]:
result = conversation.invoke(
    "Hi I'm Srijan Dubey.",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

In [42]:
result = conversation.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Your name is Srijan Dubey, as per the information provided earlier.


In [43]:
store

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi I'm Srijan Dubey.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Srijan Dubey! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-03T20:51:42.515460408Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2614985005, 'load_duration': 18737042, 'prompt_eval_count': 34, 'prompt_eval_duration': 486197000, 'eval_count': 30, 'eval_duration': 2068363000}, id='run-c1518f22-dbe6-4f6a-9af4-68ab96c876aa-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Srijan Dubey, as per the information provided earlier.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at

In [62]:
%pip install -q langchain-community SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [63]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [64]:
# Function to get session history
# create sync sql message history by connection_string
def get_session_history(session_id: str):
    return SQLChatMessageHistory(f"{session_id}", "sqlite:///memory.db")

In [65]:
config = {"configurable": {"session_id": "session1"}}

In [66]:
conversation_sql = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [67]:
result = conversation_sql.invoke(
    "Hi I'm Srijan Dubey.",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Déjà vu! You're right back at it again, Srijan Dubey! Don't worry, I'll keep track of our conversation. How's your day going so far?


In [68]:
result = conversation_sql.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: Srijan Dubey! We've been over this before. Your name is still the same. Would you like to talk about something specific or just chat for a bit?


In [69]:
config = {"configurable": {"session_id": "session2"}}

In [70]:
result = conversation_sql.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

Answer from OpenAI: I still don't have any information about your name. As I mentioned earlier, I'm a large language model, and each time you interact with me, it's a new conversation. I don't retain any information from previous conversations or keep track of individual users.

If you'd like to share your name with me, I'd be happy to chat with you!


In [73]:
# create sync sql message history by connection_string
message_history = get_session_history("session1")

In [76]:
message_history.get_messages()

[HumanMessage(content="Hi I'm Srijan Dubey.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Srijan Dubey! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-03T20:59:36.358538237Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2178131579, 'load_duration': 18295156, 'prompt_eval_count': 34, 'prompt_eval_duration': 70962000, 'eval_count': 30, 'eval_duration': 2046613000}, id='run-56b30fce-5cc2-4b1a-9925-c5b44a5828f1-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Srijan Dubey, as per the information provided earlier.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-03T20:59:52.249458608Z', 'me